# Check have I implemented the nystrom approximation speed up correctly...

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow_probability import distributions as tfd
from tensorflow_probability import bijectors as tfb
import math
import copy

from braincoder.bprf_GPs import *


In [ ]:
n_loop = 1
nvx = 10000
m = 500

x = np.linspace(0,100,nvx)*10
dXs = np.abs(x[None,:] - x[...,None])


mfunc_mean = 0.0
gpks_l = 100.0
gpks_v = 20.0
gpk_nugget = 0.1


gpN = GP(n_vx=nvx, eps=1e-3)
gpN.add_xid_stationary_kernel(
    xid='s', dXs=dXs, psd_control=None, 
)
gpN.add_nystrom_approximation(n_inducers=m)
K_true = gpN._return_sigma_full(
    gpks_l=gpks_l, gpks_v=gpks_v, gpk_nugget=gpk_nugget
)
random_params = np.random.multivariate_normal(np.zeros(nvx), K_true.numpy(), size=n_loop)
# --- Timing the Nystrom approximation loop ---
from time import time
print("Timing Nystrom approximation...")
start_nystrom = time()
No = []
for i in range(n_loop):
    eg_rand = random_params[i]
    No.append(gpN._return_log_prob_nystrom(
        parameter=tf.cast(eg_rand, tf.float32),
        mfunc_mean=mfunc_mean,
        gpks_l=gpks_l,
        gpks_v=gpks_v,
        gpk_nugget=gpk_nugget,
    ).numpy())
end_nystrom = time()
nystrom_time = end_nystrom - start_nystrom
print(f"Nystrom approximation loop took: {nystrom_time:.4f} seconds")

# --- Timing the unfixed loop ---
print("\nTiming unfixed calculation...")
start_unfixed = time()
Fo = []
for i in range(n_loop):
    # Use the same random parameter as the Nystrom loop
    eg_rand = random_params[i]
    Fo.append(gpN._return_log_prob_unfixed(
        parameter=tf.cast(eg_rand, tf.float32),
        mfunc_mean=mfunc_mean,
        gpks_l=gpks_l,
        gpks_v=gpks_v,
        gpk_nugget=gpk_nugget,
    ).numpy())
end_unfixed = time()
unfixed_time = end_unfixed - start_unfixed
print(f"Unfixed calculation loop took: {unfixed_time:.4f} seconds")

# --- Comparing the results ---
print("\n--- Comparison ---")
if nystrom_time < unfixed_time:
    print(f"The Nystrom approximation was faster by a factor of: {unfixed_time / nystrom_time:.2f}")
elif unfixed_time < nystrom_time:
    print(f"The unfixed calculation was faster by a factor of: {nystrom_time / unfixed_time:.2f}")
else:
    print("The two calculations took approximately the same amount of time.")
No, Fo = np.array(No), np.array(Fo)
# You can now calculate the correlation if you wish
correlation = np.corrcoef(No, Fo)[0, 1]
mse=(np.diff(No-Fo)**2).mean()
print(f"\nCorrelation between Nystrom and Unfixed results: corr={correlation:.4f}, mse={mse}")

In [ ]:
gpN.eps

In [ ]:
# Generate standard normal random samples


# # Transform samples using the Cholesky decomposition to get correlated samples
# random_params = np.dot(random_samples, L.T)
# import matplotlib.pyplot as plt
# plt.plot(random_samples[:,0])

In [ ]:
random_samples.shape

In [ ]:
from dpu_mini.plot_functions import dag_scatter
dag_scatter(
    No, Fo, do_id_line=True, 
)
No